---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
'''
# I am doing this to detect PA networks, which have few nodes with very high degree score 
#(important nodes) and the vast majority of nodes with few connections

for g in P1_Graphs:
    avg_clust = nx.average_clustering(g)
    dist = nx.average_shortest_path_length(g)
    degree = sorted(list(g.degree().values()))
    degree = pd.DataFrame(degree)
    v = degree[0].value_counts()
    v.sort_index(inplace=True)
    print(v)
'''

'\n# I am doing this to detect PA networks, which have few nodes with very high degree score \n#(important nodes) and the vast majority of nodes with few connections\n\nfor g in P1_Graphs:\n    avg_clust = nx.average_clustering(g)\n    dist = nx.average_shortest_path_length(g)\n    degree = sorted(list(g.degree().values()))\n    degree = pd.DataFrame(degree)\n    v = degree[0].value_counts()\n    v.sort_index(inplace=True)\n    print(v)\n'

In [5]:
def graph_identification():
    P1_Graphs = pickle.load(open('A4_graphs','rb'))
    #Prerefential Attachment : Low Distance (short paths) and Low Clustering Coeff
    #Small World - Low Rewiring : Distance very high, Clusterring Coeff very high
    #Small World - High Rewiring : Distances very small, Clustering Coeff very small
    for g in P1_Graphs:
        avg_clust = nx.average_clustering(g)
        dist = nx.average_shortest_path_length(g)
        degree = sorted(set(g.degree().values()))
        #print("clustering ", avg_clust, "\n", "avg distance ", dist, "\n\n")

    result = ['PA','SW_L','SW_L', 'PA','SW_H']
    return result

In [6]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [7]:
# calculate degree of each node - how many people send e-mails to this person? managers will have a high degree?
# managers would have high degree centrality - all members of a department will send emails to manager
# managers would have high closeness centrality

In [2]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

# Assumption - managers receive more e-mails than send

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [9]:
def salary_predictions():
    
    base = G.nodes(data=True)
    dc = nx.degree_centrality(G)
    cc = nx.closeness_centrality(G)
    bc = nx.betweenness_centrality(G)
    clust = nx.clustering(G)
    clean = list()
    lista = list(zip(base, dc.values(), cc.values(), bc.values(), clust.values()))
    for line in lista:
        clean.append([line[0][0], line[0][1]['Department'], line[0][1]['ManagementSalary'], line[1] , line[2] ,line[3], line[4]  ])


    df = pd.DataFrame(clean)
    df.columns = ['index', 'Department', 'ManagementSalary', 'dc', 'cc', 'bc', 'clust']
    df.set_index(['index'], inplace=True)
    
    y_train = df.loc[df.ManagementSalary.notnull(), ['ManagementSalary']]
    X_train = df.loc[df.ManagementSalary.notnull(), ['dc', 'cc', 'bc', 'clust']]
    X_test = df.loc[df.ManagementSalary.isnull(), ['dc', 'cc', 'bc', 'clust']]
    ind = X_test.index
    import sklearn
    # NORMALIZING
    X_train = sklearn.preprocessing.normalize(X_train)
    X_test = sklearn.preprocessing.normalize(X_test)

    from sklearn.linear_model import LogisticRegression
    classifier = LogisticRegression(C=100)
    classifier.fit(X=np.array(X_train), y=np.array(y_train) )
    result = classifier.predict_proba(np.array(X_test))

    res = [x[1] for x in result]

    return pd.Series(res, index=ind)# Your Answer Here

In [49]:
#df[df.ManagementSalary == 1].describe()

In [50]:
#df[df.ManagementSalary ==0].describe()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [3]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [48]:
#future_connections[future_connections['Future Connection'] == 0].describe()
#future_connections[future_connections['Future Connection'] == 1].describe()

In [42]:
def new_connections_predictions():
    #X_train = future_connections[future_connections['Future Connection'].notnull()]
    future_connections['common_neigh'] = future_connections.apply(lambda row: nx.common_neighbors(G, row.name[0], row.name[1]), axis=1)
    future_connections['common_neigh'] = future_connections['common_neigh'].apply(list).apply(len)

    X_train = future_connections.loc[future_connections['Future Connection'].notnull(), ['common_neigh']]
    y_train = future_connections.loc[future_connections['Future Connection'].notnull(), ['Future Connection']]
    X_test = future_connections.loc[future_connections['Future Connection'].isnull(), ['common_neigh']]


    from sklearn.linear_model import LogisticRegression
    classifier = LogisticRegression(C=100)
    classifier.fit(X=X_train.as_matrix(), y=y_train.as_matrix())
    result = classifier.predict_proba(np.array(X_test))
    #The column 0 is the probability for class 0 and the column 1 is the probability for the class 1
    res = [x[1] for x in result]

    result = pd.Series(res, index=X_test.index)
    return result